In [27]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [28]:
a = 'XY_bal_log_Rel.txt'
cols = ['E_peak', 'beta', 'sigma', 'beta_Rel', '0,1=astro,DM']

In [ ]:
# Leer el título del archivo
title = pd.read_csv(a, nrows=0, usecols=[0])
title = title.columns.values[0]
print(title)

In [ ]:
# Leer los datos del archivo
data = pd.read_csv(a, sep='\s+', names=cols, skiprows=1, header=None)
data.sample(5)